In [1]:
from get_training_data import *

import tensorflow as tf
import keras
import pickle
import gzip
import os
import random
import matplotlib.pyplot as plt
%matplotlib nbagg

/home/andy/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with gzip.open('train_cases.pkl.gz','rb') as fp:
    train_cases = pickle.load(fp)
type(train_cases), len(train_cases)

(tuple, 2)

In [3]:
train_X = train_cases[0]
train_y = train_cases[1]
ntrain = train_X.shape[0]
type(train_X), train_X.shape, train_X.dtype, type(train_y), train_y.shape, train_y.dtype

(numpy.ndarray,
 (16384, 16, 16, 128),
 dtype('float32'),
 numpy.ndarray,
 (16384, 1),
 dtype('float64'))

In [4]:
with gzip.open('valid_cases.pkl.gz','rb') as fp:
    valid_cases = pickle.load(fp)
valid_X = valid_cases[0]
valid_y = valid_cases[1]
nvalid = valid_X.shape[0]
type(valid_X), valid_X.shape, valid_X.dtype, type(valid_y), valid_y.shape, valid_y.dtype

(numpy.ndarray,
 (4096, 16, 16, 128),
 dtype('float32'),
 numpy.ndarray,
 (4096, 1),
 dtype('float64'))

In [5]:
def choose_training_case():
    i = random.randint(0, ntrain-1)
    return(train_X[i,:,:,:], train_y[i,:])

def choose_validation_case():
    i = random.randint(0, nvalid-1)
    return(valid_X[i,:,:,:], valid_y[i,:])

In [6]:
def make_batch_from_megabatch(batchsize=32, validation=False):
    get_case = choose_validation_case if validation else choose_training_case    
    xbatch = np.empty([0, INPUTS_PER_BEAT, NBEATS, NCHANNELS],dtype=np.float32)
    ybatch = np.empty([0, 1])
    for i in range(batchsize):
        case = get_case()
        x = np.expand_dims(case[0], axis=0)
        xbatch = np.concatenate([xbatch, x], axis=0)
        y = np.array(int(case[1])).reshape([1,1])
        ybatch = np.concatenate([ybatch, y], axis=0)
    return(xbatch, ybatch)

def train_gen(batchsize=32):
    while True:
        yield(make_batch_from_megabatch(batchsize, validation=False))

def valid_gen(batchsize=32):
    while True:
        yield(make_batch_from_megabatch(batchsize, validation=True))

In [7]:
# The generators are way too slow: have to fix this
for t in train_gen():
    break
t[0].shape, t[1].shape

((32, 16, 16, 128), (32, 1))

In [8]:
for v in valid_gen():
    break
v[0].shape, v[1].shape

((32, 16, 16, 128), (32, 1))

In [9]:
input_shape = (INPUTS_PER_BEAT, NBEATS, NCHANNELS)

model = keras.Sequential()
model.add(keras.layers.Conv2D(64, 1, activation='relu', input_shape=input_shape))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(32, 1, activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(16, (2,4), activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(25, 5, activation='relu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(4, 3, activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dropout(.5))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [10]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 64)        8256      
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        2080      
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 13, 16)        4112      
_________________________________________________________________
batch_normalization_3 (Batch (None, 15, 13, 16)        64        
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 9, 25)         10025     
__________

In [12]:
valid=valid_gen()
train=train_gen()
model.fit_generator(train, 
                    validation_data=valid, validation_steps=64,
                    epochs=60, 
                    steps_per_epoch=64)

Epoch 1/60
64/64 [==============================] - 2s 31ms/step - loss: 0.6561 - acc: 0.6904 - val_loss: 0.6319 - val_acc: 0.7280
Epoch 2/60
64/64 [==============================] - 1s 13ms/step - loss: 0.5931 - acc: 0.7417 - val_loss: 0.5959 - val_acc: 0.7329
Epoch 3/60
64/64 [==============================] - 1s 13ms/step - loss: 0.5672 - acc: 0.7568 - val_loss: 0.5445 - val_acc: 0.7666
Epoch 4/60
64/64 [==============================] - 1s 13ms/step - loss: 0.5392 - acc: 0.7681 - val_loss: 0.5402 - val_acc: 0.7725
Epoch 5/60
64/64 [==============================] - 1s 14ms/step - loss: 0.5195 - acc: 0.7812 - val_loss: 0.5527 - val_acc: 0.7510
Epoch 6/60
64/64 [==============================] - 1s 13ms/step - loss: 0.5553 - acc: 0.7534 - val_loss: 0.5719 - val_acc: 0.7729
Epoch 7/60
64/64 [==============================] - 1s 13ms/step - loss: 0.5198 - acc: 0.7661 - val_loss: 0.5150 - val_acc: 0.7705
Epoch 8/60
64/64 [==============================] - 1s 13ms/step - loss: 0.5131 - a

In [13]:
for i in range(10):
    song, tempo, compatible, clip = get_validation_case()
    print( song, tempo )
    c = clip_to_tf_input(resample_clip(clip))
    x = np.expand_dims(c, axis=0)
    p = model.predict(x)[0][0]
    print( 'Predicted: ', p, '   Actual: ', compatible, '\n' )

Kalamazoo 185
Predicted:  0.0063344482    Actual:  False 

09 Suddenly I See 77
Predicted:  1.036702e-05    Actual:  False 

IllBeThereForYou 191
Predicted:  0.7664778    Actual:  True 

AHardRainsAGonnaFall 47
Predicted:  0.06766578    Actual:  False 

HoldMeTight 42
Predicted:  0.0146520445    Actual:  False 

IThinkWereAloneNow2MP 53
Predicted:  4.1587764e-06    Actual:  False 

IfIHadAHammer 47
Predicted:  2.217749e-05    Actual:  False 

ChatanoogaChooChoo 105
Predicted:  0.00042409147    Actual:  False 

BothSidesNow 42
Predicted:  6.157358e-05    Actual:  False 

TheBollWeevil 42
Predicted:  0.1895164    Actual:  False 

